In [1]:
%pip install crewai 'crewai[tools]'


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
import yaml
from crewai import Agent, Task, Crew , LLM , Process
from crewai.tools import tool

In [13]:
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [4]:
llm = LLM(
    model="ollama/llama3.1:latest",
    base_url="http://localhost:11434"
)

In [27]:
from crewai_tools import FileWriterTool

# Initialize the tool
file_writer_tool = FileWriterTool() # to write modifications in new pdf

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style

In [14]:
code_writer_agent=Agent(
    config=agents_config['code_writer'],
    llm=llm
)

In [29]:
code_executor_agent=Agent(
    config=agents_config['code_executor'],
    llm=llm,
    tools=[file_writer_tool],
    allow_code_execution=True
)

In [11]:
print(tasks_config['modify_file_task'])


{'description': 'Read the file located at {file_path} and make the following modifications: {modifications}.   Ensure the generated Python code is clean, follows best practices, and is free of errors.   The output should be a complete Python script that can be executed without issues. The Python script should have code for saving the file after modifications.The modifications should be combined with the original file content.\n', 'expected_output': 'A well-structured Python script that correctly implements the required modifications   to the file at {file_path}, ensuring efficiency and error handling. The Python script should have code for saving the file after modifications.The modifications should be combined with the original file content.\n', 'agent': 'code_writer'}


In [12]:
print(type(tasks_config['modify_file_task']))


<class 'dict'>


In [30]:
code_genration_task=Task(
    config=tasks_config['modify_file_task'],
    agent=code_writer_agent
)

In [31]:
code_execution=Task(
    config=tasks_config['execute_and_save_task'],
    agent=code_executor_agent
)

In [32]:
main_crew = Crew(
    agents=[code_writer_agent,code_executor_agent],
    tasks=[code_genration_task,code_execution],
    process=Process.sequential,
    verbose=True,

)

In [33]:
inputs = {
        'file_path': '/Users/saish/Downloads/AutoFilePOC/nihilentpoc/src/nihilentpoc/word-file.pdf',
        'modifications': "Make changes to the first table in the file by adding random names to it with gender"
    }

In [34]:
print(type(inputs)) 

<class 'dict'>


In [35]:
main_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3063b9de-c745-4ac9-b66c-6a97ad4ee129                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 379a3210-260b-4fea-beaa-d74b430cf2c9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 379a3210-260b-4fea-beaa-d74b430cf2c9
       Status: Executing Task...
    └── 🤖 Agent: Senior Python Developer
        
            Status: In Progress

# Agent: Senior Python Developer
## Task: Read the file located at /Users/saish/Downloads/AutoFilePOC/nihilentpoc/src/nihilentpoc/word-file.pdf and make the following modifications: Make changes to the first table in the file by adding random names to it with gender.   Ensure the generated Python code is clean, follows best practices, and is free of errors.   The output should be a complete Python script that can be executed without issues. The Python script should have code for saving the file after modifications.The modifications should be combined with the original file content.



🚀 Crew: crew
└── 📋 Task: 379a3210-260b-4fea-beaa-d74b430cf2c9
       Status: Executing Task...
    └── 🤖 Agent: Senior Python Developer
        
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 379a3210-260b-4fea-beaa-d74b430cf2c9
│      Assigned to: Senior Python Developer
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Python Developer
│       
│           Status: ✅ Completed
└── 📋 Task: fff0672b-ba34-4360-8f21-934c30aa6a97
       Status: Executing Task...
    └── 🤖 Agent: Python Code Execution Agent
        
            Status: In Progress
        └── 🔧 Using Code Interpreter (2)

🚀 Crew: crew
├── 📋 Task: 379a3210-260b-4fea-beaa-d74b430cf2c9
│      Assigned to: Senior Python Developer
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Python Developer
│       
│           Status: ✅ Completed
└── 📋 Task: fff0672b-ba34-4360-8f21-934c30aa6a97
       Status: Executing Task...
    └── 🤖 Agent: Python Code Execution Agent
        
            Status: In Progress
        └── 🔧 Used Code Interpreter (2)



# Agent: Python Code Execution Agent
## Using tool: Code Interpreter
## Tool Input: 
"{\"code\": \"the actual python script\", \"libraries_used\": [\"PyPDF2\", \"fpdf\", \"pandas\", \"numpy\"]}"
## Tool Output: 
Something went wrong while running the code: 
  File "<string>", line 1
    the actual python script
        ^^^^^^
SyntaxError: invalid syntax



🚀 Crew: crew
├── 📋 Task: 379a3210-260b-4fea-beaa-d74b430cf2c9
│      Assigned to: Senior Python Developer
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Python Developer
│       
│           Status: ✅ Completed
└── 📋 Task: fff0672b-ba34-4360-8f21-934c30aa6a97
       Status: Executing Task...
    └── 🤖 Agent: Python Code Execution Agent
        
            Status: In Progress
        ├── 🔧 Used Code Interpreter (2)
        └── 🧠 Thinking...

KeyboardInterrupt: 

In [ ]:
# add code interpretor tool instead of allow_code_execution as we need to install librabires in it "{\"code\": \"the actual python script\", \"libraries_used\": [\"PyPDF2\", \"fpdf\", \"pandas\", \"numpy\"]}"
# also modify tasks for better results